In [1]:
import numpy as np
import pandas as pd 
import matplotlib.pyplot as plt
import seaborn as sns
import sklearn
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_absolute_error
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OrdinalEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import OneHotEncoder
from sklearn.model_selection import cross_val_score

%matplotlib inline


import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/home-data-for-ml-course/sample_submission.csv
/kaggle/input/home-data-for-ml-course/sample_submission.csv.gz
/kaggle/input/home-data-for-ml-course/train.csv.gz
/kaggle/input/home-data-for-ml-course/data_description.txt
/kaggle/input/home-data-for-ml-course/test.csv.gz
/kaggle/input/home-data-for-ml-course/train.csv
/kaggle/input/home-data-for-ml-course/test.csv


In [2]:
train_df = pd.read_csv('/kaggle/input/home-data-for-ml-course/train.csv', index_col='Id')

train_df.head()

,MSSubClass,MSZoning,LotFrontage,LotArea,Street,Alley,LotShape,LandContour,Utilities,LotConfig,...,PoolArea,PoolQC,Fence,MiscFeature,MiscVal,MoSold,YrSold,SaleType,SaleCondition,SalePrice
Id,,,,,,,,,,,,,,,,,,,,,
1,60,RL,65.0,8450,Pave,NaN,Reg,Lvl,AllPub,Inside,...,0,NaN,NaN,NaN,0,2,2008,WD,Normal,208500
2,20,RL,80.0,9600,Pave,NaN,Reg,Lvl,AllPub,FR2,...,0,NaN,NaN,NaN,0,5,2007,WD,Normal,181500
3,60,RL,68.0,11250,Pave,NaN,IR1,Lvl,AllPub,Inside,...,0,NaN,NaN,NaN,0,9,2008,WD,Normal,223500
4,70,RL,60.0,9550,Pave,NaN,IR1,Lvl,AllPub,Corner,...,0,NaN,NaN,NaN,0,2,2006,WD,Abnorml,140000
5,60,RL,84.0,14260,Pave,NaN,IR1,Lvl,AllPub,FR2,...,0,NaN,NaN,NaN,0,12,2008,WD,Normal,250000


In [3]:
train_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1460 entries, 1 to 1460
Data columns (total 80 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   MSSubClass     1460 non-null   int64  
 1   MSZoning       1460 non-null   object 
 2   LotFrontage    1201 non-null   float64
 3   LotArea        1460 non-null   int64  
 4   Street         1460 non-null   object 
 5   Alley          91 non-null     object 
 6   LotShape       1460 non-null   object 
 7   LandContour    1460 non-null   object 
 8   Utilities      1460 non-null   object 
 9   LotConfig      1460 non-null   object 
 10  LandSlope      1460 non-null   object 
 11  Neighborhood   1460 non-null   object 
 12  Condition1     1460 non-null   object 
 13  Condition2     1460 non-null   object 
 14  BldgType       1460 non-null   object 
 15  HouseStyle     1460 non-null   object 
 16  OverallQual    1460 non-null   int64  
 17  OverallCond    1460 non-null   int64  
 18  YearBuil

 # Model training with dropping object columns and columns with missing values

In [4]:
# train and validation data set 

y = train_df['SalePrice']

train_df2 = train_df.drop(['SalePrice'], axis=1)
X = train_df2.select_dtypes(exclude=['object'])

X_train, X_valid, y_train, y_valid = train_test_split(X,y, train_size=0.8, test_size=0.2, random_state=0)

col_wiht_missing_value = [col for col in X_train.columns 
                          if X_train[col].isnull().any()]

print(col_wiht_missing_value)

X_train = X_train.drop(col_wiht_missing_value, axis=1)
X_valid = X_valid.drop(col_wiht_missing_value, axis=1)

# random forest model training

model = RandomForestRegressor(random_state=0)

model.fit(X_train,y_train)

# predicting validation values

pred_values = model.predict(X_valid)

# Calculating MAE

MAE = mean_absolute_error(y_valid,pred_values)

print("mean absolute error of model:", MAE)

['LotFrontage', 'MasVnrArea', 'GarageYrBlt']
mean absolute error of model: 17837.82570776256


In [5]:
test_df = pd.read_csv('/kaggle/input/home-data-for-ml-course/test.csv', index_col=0)

test_df.head()

,MSSubClass,MSZoning,LotFrontage,LotArea,Street,Alley,LotShape,LandContour,Utilities,LotConfig,...,ScreenPorch,PoolArea,PoolQC,Fence,MiscFeature,MiscVal,MoSold,YrSold,SaleType,SaleCondition
Id,,,,,,,,,,,,,,,,,,,,,
1461,20,RH,80.0,11622,Pave,NaN,Reg,Lvl,AllPub,Inside,...,120,0,NaN,MnPrv,NaN,0,6,2010,WD,Normal
1462,20,RL,81.0,14267,Pave,NaN,IR1,Lvl,AllPub,Corner,...,0,0,NaN,NaN,Gar2,12500,6,2010,WD,Normal
1463,60,RL,74.0,13830,Pave,NaN,IR1,Lvl,AllPub,Inside,...,0,0,NaN,MnPrv,NaN,0,3,2010,WD,Normal
1464,60,RL,78.0,9978,Pave,NaN,IR1,Lvl,AllPub,Inside,...,0,0,NaN,NaN,NaN,0,6,2010,WD,Normal
1465,120,RL,43.0,5005,Pave,NaN,IR1,HLS,AllPub,Inside,...,144,0,NaN,NaN,NaN,0,1,2010,WD,Normal


In [6]:
X = test_df.select_dtypes(exclude=['object'])

X_test = X.drop(col_wiht_missing_value, axis=1)

X_test.dropna(inplace=True)

test_pred = model.predict(X_test)

test_pred[:5]

my_submission = pd.DataFrame({'id': X_test.index,
                              'SalePrice': test_pred})

my_submission.to_csv('submission.csv', index=False)

# Ordinal encoding of categorical variables

In [7]:
train_df = pd.read_csv('/kaggle/input/home-data-for-ml-course/train.csv', index_col='Id')

y = train_df['SalePrice']

X = train_df.drop(['SalePrice'], axis=1)


# dividing train and validation data

X_train, X_valid, y_train, y_valid = train_test_split(X, y, train_size=0.8,
                                                      test_size=0.2, random_state=0)

X_train = X_train.drop(col_wiht_missing_value, axis=1)
X_valid = X_valid.drop(col_wiht_missing_value, axis=1)


# removing noisy categorical columns

object_cols = [col for col in X_train.columns if X_train[col].dtype == "object"]


good_labels = [col for col in object_cols if set(X_valid[col]).issubset(set(X_train[col]))]

bad_labels = list(set(object_cols)-set(good_labels))

print(bad_labels)

X_train_label = X_train.drop(bad_labels, axis=1)
X_valid_label = X_valid.drop(bad_labels, axis=1)

categorical_missing = [col for col in good_labels if X_train[col].isnull().any()]

print(categorical_missing)

X_train_label = X_train_label.drop(categorical_missing, axis=1)
X_valid_label = X_valid_label.drop(categorical_missing, axis=1)

cat_objects = list(set(good_labels)-set(categorical_missing))

# encoding categorical variables 

myencoder = OrdinalEncoder()

X_train_label[cat_objects] = myencoder.fit_transform(X_train_label[cat_objects])
X_valid_label[cat_objects] = myencoder.transform(X_valid_label[cat_objects])

# fitting model to data

mymodel = RandomForestRegressor(n_estimators=100, random_state=0)

mymodel.fit(X_train_label,y_train)

predicted_val = mymodel.predict(X_valid_label)

# calculating MAE of the model

MAE = mean_absolute_error(y_valid,predicted_val)

print("mean absolute error of mymodel: ", MAE)

['MiscFeature', 'Condition2', 'Functional', 'RoofMatl']
['Alley', 'MasVnrType', 'BsmtQual', 'BsmtCond', 'BsmtExposure', 'BsmtFinType1', 'BsmtFinType2', 'Electrical', 'FireplaceQu', 'GarageType', 'GarageFinish', 'GarageQual', 'GarageCond', 'PoolQC', 'Fence']
mean absolute error of mymodel:  17098.01649543379


# Using sklearn pipeline

In [8]:
# importing data sets

y = train_df['SalePrice']

X = train_df.drop(['SalePrice'], axis=1)

X_train, X_valid, y_train, y_valid = train_test_split(X,y, train_size=0.8, test_size=0.2,
                                                     random_state=0)
# categorical columns

categorical_cols = [col for col in X_train.columns if X_train[col].dtype == 'object' and 
                    X_train[col].nunique() < 9]

# numerical columns

numerical_cols = [col for col in X_train.columns if X_train[col].dtype in ['int64','float64']]
    
# total number of columns to be learn from 

my_cols = categorical_cols + numerical_cols

train_X =X_train[my_cols].copy()
valid_X = X_valid[my_cols].copy()
test_X = test_df[my_cols].copy()

print("Total number of columns to preprocess: ", len(my_cols))

Total number of columns to preprocess:  74


In [9]:
# numerical columns transoformation
numerical_transformer = SimpleImputer(strategy='mean')

# categorical columns transformation

categorical_transformer = Pipeline(steps=[('impute', SimpleImputer(strategy='constant')),
                                          ('onehot', OneHotEncoder(handle_unknown='ignore', sparse=False))
                             ])

# preprocess both num and cat columns

preprocessor = ColumnTransformer(transformers=[('num', numerical_transformer, numerical_cols),
                                               ('cat', categorical_transformer,categorical_cols)
])

# define model

model_ = RandomForestRegressor(n_estimators=155, random_state=0)

# combine both preprocessor and model

final_model = Pipeline(steps=[('preprocessor', preprocessor),
                              ('model',model_ )
    ])

# fitting model to data

final_model.fit(train_X, y_train)

# predcting outcomes

predicted_price = final_model.predict(valid_X)

# calculating MAE

MAE = mean_absolute_error(y_valid,predicted_price)

print("MAE: ", MAE)

MAE:  17266.14752540875


In [10]:
test_pred = final_model.predict(test_X)


mysub = pd.DataFrame({'Id': test_X.index,
                      'SalePrice': test_pred})

mysub.to_csv('submission.csv', index=False)

> Considering above two model ordinal encoding has lower MAE than onehot encoding

# Cross validation having cv fold = 5

In [11]:
# using cross validation on dataset

y = train_df['SalePrice']

train_df2 = train_df.drop(['SalePrice'], axis=1)

numerical_cols = [col for col in train_df2.columns if train_df2[col].dtype in ['int64','float64']]

X = train_df2[numerical_cols]

# building model and imputer pipeline

mypipeline = Pipeline(steps=[('imputer', SimpleImputer(strategy='mean')),
                             ('model', RandomForestRegressor(n_estimators=200
                                                            , random_state=0))])

# cross validation scoring 

scores = -1*cross_val_score(mypipeline, X,y, cv=5, scoring='neg_mean_absolute_error')

print("average of mean absolute error is: ", scores.mean())

average of mean absolute error is:  17937.141941780825


# Thank you!